# 作业

-----


## 1.0 立体角模拟

**采用 Steping 数据输出**

- 设置探测器
    - 粒子源位于原点 （0，0，0）
    - 环形探测器 2 个（型号S13），灵敏面积内径 $\Phi = 25mm$，外径 $\Phi = 121mm$，厚度 300um
    - 矩形探测器 3 个（型号BB30），灵敏面积长96mm，宽 64mm，厚度 300um
    - 2个环形探测器距离原点均为 5.0 cm,三个矩形探测器距离原点均为 6.5 cm
    - 摆放位置如下图所示
- 将材料设置为真空
- 选择一个粒子，4$\pi$ 均匀发射
- 模拟给出 5 个探测器的立体角和总立体角

![hw1](img/hw1.png)



-----


## 2.0 $\beta$ 探测效率模拟

**采用 Steping 数据输出**

- 硅探测器
    - 型号 W1，灵敏面积 50x50 mm$^2$, 厚度 300 um 
- 假设 $\beta$("e-") 在探测器某侧表面均匀分布
-  $\beta$ 发射方向为朝向探测器一侧 2$\pi$ 均匀分布
- 模拟给出“能量-效率”曲线（横坐标为$\beta$能量，纵坐标为探测效率）
    - 模拟给出 100，150, 200，300，500，700，1000, 1500, 2000 keV $\beta$的探测效率
        - 例如，对于 450 keV 能点，模拟 1000000 个事件，数据处理得到全能峰事件为 240351 个，则 该能量下的探测效率为 24.0%


![hw2](img/hw2.png)


-----

## 3.0 移除不必要的数据输出

**采用Steping 数据输出**

- 在 2.0 $\beta$ 探测效率模拟中，默认的数据输出中有 40 个 Branch，其中绝大多数 Branch 在后期的数据分析中是多余的
- 修改 wuRunActionAll 和 wuSteppingActionAll 两个类，删除多余的 Branch，使输出的数据尽可能小。
- 参考 [RunAction](RunAction.html) 中代码，传递运行参数。
- 重复模拟作业 2.0 中的“能量-效率”曲线，需给出每个点的误差。


```cpp
// 如何将多个 ROOT 文件 串起来处理，供参考
TChain *tree = new TCahin("t");//“t” 是 ROOT 文件中 TTree 的名字

// 采用通配符示例 1
tree->Add("*.root");
// 采用通配符示例 2
tree->Add("data*.root");

// 逐个文件添加
tree->Add("data0002_t0.root");
tree->Add("data0002_t1.root");
tree->Add("data0003_t0.root");
tree->Add("data0003_t1.root");


// tree->Process();
```


-----

## 4.0 角分布模拟

**采用 Steping 数据输出**

- 双面硅探测器 
    - 厚度分别为 140 um，40 um，300 um，依次排列
    - 140 um 与 40 um 硅中心距离 10 mm，40 um 与 300 um 硅中心距离 15 mm
    - 型号 W1，灵敏面积 50x50 mm$^2$，正背面各分成 16 条
    - 模拟考虑像素颗粒度，每个探测器分成 16$\times$16 个 3.125$\times$3.125 mm$^2$ 小单元进行建模和数据处理
- 假设在 40 um 探测器上发生某原子核的双质子衰变（从粒子源发射出两个质子）
    - 衰变位置位于 40 um 硅上均匀分布(x/y/z 三个维度)
    - 质子总能量为 5.0 MeV，随机分配给两个质子
    - 每个质子独立 4$\pi$ 均匀发射
- 模拟给出探测到的两个质子的角分布
    - 横坐标是角度，[0,180] 度，纵坐标是计数
    - 双质子均位于 140 um 硅探测器，双质子均位于 300 um 硅探测器，一个质子位于 140 um、一个位于 300 um 硅探测器
    - 给出以上三种情况夏各自的角分布以及总角分布


**提示： 实际探测中，只能知道粒子打在哪个像素中，不能得到像素中的具体位置。因此数据处理中，计算角度时候，需要考虑这个因素！！！**


-----

## 5.0 调用外部 ROOT

*（选作）*

- 利用以下代码生成衰变位置在硅上的分布信息
    - TH2 为 16$\times$16 的位置信息
    - TH1 为注入深度分布信息（0-40 um）
- 采用 TH2/TH1 分布抽用来产生作业 4.0 中的衰变位置
- 重复作业 4.0 中的模拟，并将结果画在一起进行对比


```cpp
// 根据以下代码生成衰变位置在硅上的分布

{
  TH2I *h2 = new TH2I("h2","",16, 0, 16, 16, 0, 16);
  TH1I *h1 = new TH1I("h1","",400, 0, 40);

  double gausx, gausy;
  double dep;
  
  for (int i = 0; i < 1000000; ++i)
    {
      gausx = gRandom->Gaus(10, 3);
      gausy = gRandom->Gaus(8, 4);
      
      if(gausx >=0 && gausx <= 16 && gausy >=0 && gausy <= 16)
	h2->Fill(gausx, gausy);

      dep = gRandom->Landau(8,10);
      if(dep >= 0 && dep <= 40)
	h1->Fill(dep);
    }

  TCanvas *c1 = new TCanvas("c1");
  c1->Divide(2,1);
  c1->cd(1);
  h2->Draw("colz");
  c1->cd(2);
  h1->Draw();
  
  TFile *filew = new TFile("dis.root","RECREATE");//"RECREATE" "READ"
  if(!filew->IsOpen())
    {
      std::cout<<"Can't open root file"<<std::endl;
    }
  h2->Write();
  h1->Write();
  filew->Close();
}
```

![hw5](img/rootsource.png)



-----

## 6.0 灵敏探测器

**采用 SD/event 数据输出**

采用灵敏探测器方式，重复作业 2.0 中的要求。


-----

## 7.0 gamma 多普勒修正

**采用 Steping 数据输出**

- 重复 [PhotoEvaporation](PhotoEvaporation.html) 中 60Ni 的退激模拟，60Ni 处于激发态，粒子能量 2 MeV，在 (0,0,0) 点各向同性发射。
- 逐事件根据 60Ni 初始粒子的能量，xMomentumDirectionPre、yMomentumDirectionPre、zMomentumDirectionPre 信息，对测量的 gamma 能量进行多普勒修正，对比修正前后的能谱
- 模拟事件数 1000000


-----

## 8.0 修改衰变分支比

**采用 Steping 数据输出**

- 参考 [RadioactiveDecay](RadioactiveDecay.html) 中的内容，*152Eu* 处于基态，粒子能量为 0 MeV，在 (0,0,0) 点。
- 修改 *152Eu* beta 衰变到子核的分支比
- 比较修改前后， Ge 探测到的 gamma 能谱
- 模拟事件数 10000000

```
# 数据库中 beta 衰变分支
                              BetaMinus      344.279  -       8.2466      1474.64
                              BetaMinus     755.3961  -      0.91363     1063.523
                              BetaMinus      930.545  -      0.28388      888.374
                              BetaMinus     1109.202  -      0.26389      709.717
                              BetaMinus     1123.186  -       13.734     695.7333
                              BetaMinus     1282.246  -     0.023191      536.673
                              BetaMinus     1318.414  -     0.018892      500.505
                              BetaMinus      1434.02  -        2.431      384.899
                              BetaMinus      1550.16  -     0.045581      268.759
                              BetaMinus     1605.609  -      0.10676       213.31
                              BetaMinus     1643.418  -       1.8323      175.501
                              BetaMinus      1692.42  -     0.019892      126.499
```


-----

## x.0 统计中子多重性

（选作）
**待补充**



-----


## 参考代码

### 立体角模拟

```cpp
// wuDetectorConstruction.hh
  G4LogicalVolume* logicBB30[3];
  G4VPhysicalVolume* physBB30[3];
  G4LogicalVolume* logicS13[2];
  G4VPhysicalVolume* physS13[2];
```

```cpp
// wuDetectorConstruction.cc

  G4double BB30X = 64*mm;
  G4double BB30Y = 96*mm;
  G4double BB30Z = 300.0*um;
  G4double S13R1 = 12.5*mm;
  G4double S13R2 = 60.5*mm;
  G4double S13Z = 300.0*um;


  G4Box* solidBB30 =    
    new G4Box("BB30",                    //its name
  	      0.5*BB30X, 0.5*BB30Y, 0.5*BB30Z); //its size

  logicBB30[0] =
    new G4LogicalVolume(solidBB30,            //its solid
  			dssd_mat,             //its material
  			"BB30_0");         //its name

  logicBB30[1] =
    new G4LogicalVolume(solidBB30,            //its solid
  			dssd_mat,             //its material
  			"BB30_1");         //its name

  logicBB30[2] =
    new G4LogicalVolume(solidBB30,            //its solid
  			dssd_mat,             //its material
  			"BB30_2");         //its name
  

  G4RotationMatrix* rotbb30_0 = new G4RotationMatrix();
  rotbb30_0->rotateY(90.0*deg);

  G4RotationMatrix* rotbb30_1 = new G4RotationMatrix();
  rotbb30_1->rotateY(90.0*deg);

  G4RotationMatrix* rotbb30_2 = new G4RotationMatrix();
  rotbb30_2->rotateY(90.0*deg);
  rotbb30_2->rotateX(90.0*deg);
  
  physBB30[0] =
    new G4PVPlacement(rotbb30_0,                       //no rotation set 0
  		      G4ThreeVector(6.5*cm,0.0,0.0), //at (0,0,0)
  		      logicBB30[0],               //its logical volume
  		      "BB30_0",              //its name
  		      logicWorld,              //its mother  volume
  		      false,                   //no boolean operation
  		      0,                       //copy number
  		      checkOverlaps);          //overlaps checking

  physBB30[1] =
    new G4PVPlacement(rotbb30_1,                       //no rotation set 0
  		      G4ThreeVector(-6.5*cm,0.0,0.0), //at (0,0,0)
  		      logicBB30[1],               //its logical volume
  		      "BB30_1",              //its name
  		      logicWorld,              //its mother  volume
  		      false,                   //no boolean operation
  		      0,                       //copy number
  		      checkOverlaps);          //overlaps checking

  physBB30[2] =
    new G4PVPlacement(rotbb30_2,                       //no rotation set 0
  		      G4ThreeVector(0.0,6.5*cm,0.0), //at (0,0,0)
  		      logicBB30[2],               //its logical volume
  		      "BB30_2",              //its name
  		      logicWorld,              //its mother  volume
  		      false,                   //no boolean operation
  		      0,                       //copy number
  		      checkOverlaps);          //overlaps checking

  
  
  G4Tubs* solidS13 =
    new G4Tubs("S13",
  	       S13R1,//内半径
  	       S13R2,//外半径
  	       0.5*S13Z,//Z轴方向的半长度
  	       0*degree,//圆周起始位置弧度值
  	       360*degree);//该实体的圆心角弧度值

  logicS13[0] =
    new G4LogicalVolume(solidS13,            //its solid
  			dssd_mat,             //its material
  			"S13_0");         //its name

  logicS13[1] =
    new G4LogicalVolume(solidS13,            //its solid
  			dssd_mat,             //its material
  			"S13_1");         //its name

  physS13[0] =
    new G4PVPlacement(0,                       //no rotation set 0
  		      G4ThreeVector(0.0,0.0,-5.0*cm), //at (0,0,0)
  		      logicS13[0],               //its logical volume
  		      "S13_0",              //its name
  		      logicWorld,              //its mother  volume
  		      false,                   //no boolean operation
  		      0,                       //copy number
  		      checkOverlaps);          //overlaps checking
      
  physS13[1] =
    new G4PVPlacement(0,                       //no rotation set 0
  		      G4ThreeVector(0.0,0.0,5.0*cm), //at (0,0,0)
  		      logicS13[1],               //its logical volume
  		      "S13_1",              //its name
  		      logicWorld,              //its mother  volume
  		      false,                   //no boolean operation
  		      0,                       //copy number
  		      checkOverlaps);          //overlaps checking
```




**征集示例需求**

In [1]:
!date
!jupyter nbconvert homework.ipynb --to html
!jupyter nbconvert homework.ipynb --to html

2024年 05月 03日 星期五 19:26:42 CST

[NbConvertApp] Converting notebook homework.ipynb to html

[NbConvertApp] Writing 600112 bytes to homework.html

[NbConvertApp] Converting notebook homework.ipynb to html


[NbConvertApp] Writing 600112 bytes to homework.html


